# Imports

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
from PIL import Image
import numpy as np #numpy
import seaborn as sns
import tensorflow as tf
# import tensorflow.compat.v2 as tf #use tensorflow v2 as a main
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import normalize # normalization of the matrix
import scipy
import pandas as pd
from tensorflow.keras.models import load_model, Model, Sequential
from tensorflow.keras import Model, layers
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Concatenate, Dense, Flatten, Conv2DTranspose
from tensorflow.keras import backend as K
from tensorflow.keras.applications import ResNet50, EfficientNetB0

import os
import cv2
from tensorflow.keras.utils import normalize,Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

import segmentation_models as sm

tf.version.VERSION

**Mix loss in section name** = mixed_loss_l2 <br>
**Advanced/another Mix loss in section name** = mixed_loss_l1

# Utility functions

In [ ]:
def data_generator(images, batch_size):
    num_samples = len(images)
    while True:
        indices = np.random.permutation(num_samples)
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices = indices[start:end]
            batch_images = images[batch_indices]
            yield batch_images, batch_images

    
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5,          
    patience=5,          
    min_lr=1e-7,         
    verbose=1         
)

class ShowReconstruction(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
      rand_id = np.random.randint(len(source_images))
      source_image = source_images[rand_id][np.newaxis,...]
      reconstructed = self.model.predict(source_image)
      real_image = target_images[rand_id]

      plt.subplot(1,3,1)
      plt.imshow(source_image[0], cmap='gray')
      plt.title("Source Image")
      plt.axis('off')

      plt.subplot(1,3,2)
      plt.imshow(reconstructed[0], cmap='gray')
      plt.title("Produced Image")
      plt.axis('off')

      plt.subplot(1,3,3)
      plt.imshow(real_image, cmap='gray')
      plt.title("Real Image")
      plt.axis('off')

      plt.tight_layout()
      #plt.savefig("Reconstruction_Epoch_{}".format(epoch))
      plt.show()

## Loss functions

In [ ]:
#MS-SSIM
def ms_ssim_loss(y_true, y_pred, max_val=1.0):
    # You don't need to transpose for grayscale images
    y_true = tf.image.convert_image_dtype(y_true, tf.float32)
    y_pred = tf.image.convert_image_dtype(y_pred, tf.float32)
    ms_ssim = 1.0 - tf.image.ssim_multiscale(y_true, y_pred, max_val=max_val)
    return ms_ssim

In [ ]:
#Loss of MSE + MS-SSIM
def mixed_loss_l2(y_true, y_pred, alpha=0.84, max_val=1.0):
    y_true = tf.image.convert_image_dtype(y_true, tf.float32)
    y_pred = tf.image.convert_image_dtype(y_pred, tf.float32)
    
    mse_loss = tf.keras.losses.MeanSquaredError()(y_true, y_pred)
    ms_ssim = 1.0 - tf.image.ssim_multiscale(y_true, y_pred, max_val=max_val)
    
    mix = alpha * ms_ssim + (1-alpha) * mse_loss
    return mix

def mixed_loss_l1(y_true, y_pred, max_val=1.0):
    mae_loss = tf.keras.losses.MeanAbsoluteError()(y_true, y_pred)
    return 0.16 * mae_loss + 0.84 * (1.0 - tf.image.ssim_multiscale(y_true, y_pred, max_val=max_val))

## Metrics

In [ ]:
def ms_ssim_metric(y_true, y_pred):
    return tf.image.ssim_multiscale(y_true, y_pred, max_val=tf.reduce_max(y_true))

In [ ]:
from tensorflow.keras import backend as K
def psnr(y_true, y_pred):
    max_pixel = 1.0
    mse = K.mean(K.square(y_true - y_pred))
    psnr = 10.0 * K.log((max_pixel**2) / mse) / K.log(10.0)
    return psnr

# Load the images

In [ ]:
source_folder = 'Dataset/source'
target_folder = 'Dataset/target'

# List all image file names in the source folder
source_image_files = os.listdir(source_folder)

# Initialize lists to store the preprocessed images
source_images = []
target_images = []

i = 0

# Loop through each image file in the source folder
for filename in source_image_files:

     # Load source image
    source_image = cv2.imread(os.path.join(source_folder, filename), cv2.IMREAD_GRAYSCALE)
    source_image = source_image.astype('float32') / 255.0
    #source_image = (source_image.astype(np.float32) - 127.5) / 127.5
    #source_image = np.repeat(source_image[:, :, np.newaxis], 3, axis=2)
    source_image = cv2.resize(source_image, (512, 512))
    source_images.append(source_image)

    # Load corresponding target image from the target folder
    target_image = cv2.imread(os.path.join(target_folder, filename), cv2.IMREAD_GRAYSCALE)
    target_image = target_image.astype('float32') / 255.0
    #target_image = (target_image.astype(np.float32) - 127.5) / 127.5
    #target_image = np.repeat(target_image[:, :, np.newaxis], 3, axis=2)
    target_image = cv2.resize(target_image, (512, 512))
    target_images.append(target_image)
    
    
    i += 1
    if i == 500:
        break

# Convert the lists to NumPy arrays
source_images = np.array(source_images)
target_images = np.array(target_images)

source_images = np.expand_dims(source_images, axis=-1)
target_images = np.expand_dims(target_images, axis=-1)
# Print the shape of the loaded and preprocessed images
print("Source Images Shape:", source_images.shape)
print("Target Images Shape:", target_images.shape)

In [ ]:
# Use first 100 as a test set
X_train = source_images[100:]
y_train = target_images[100:]

X_val = source_images[:100]
y_val = target_images[:100]

print(f"x_train: {len(X_train)} | x_val: {len(X_val)} | y_train: {len(y_train)} | y_val: {len(y_val)}")

In [ ]:
# For the whole dataset, the DataGenerator needs to be used
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [ ]:
# Kaggle dataset test
test_folder = 'Dataset/test/'

source_image_files = os.listdir(test_folder)

source_image_files = [filename for filename in source_image_files if filename.lower().endswith('.png')]
   
test_images = []

i = 0
for filename in source_image_files:
    test_image = cv2.imread(os.path.join(test_folder, filename), cv2.IMREAD_GRAYSCALE)
    test_image = cv2.resize(test_image, (1024, 1024))
    #test_image = cv2.bitwise_not(test_image)
    test_image = test_image.astype('float32') / 255.0
    test_images.append(test_image)
    if i == 4:
        break
    i += 1

test_images = np.array(test_images)
test_images = np.expand_dims(test_images, axis=-1)
print("Test Images Shape:", test_images.shape)

In [ ]:
# Provided images from Olomouc
test_folder = 'Test/'

source_image_files = os.listdir(test_folder)

source_image_files = [filename for filename in source_image_files if filename.lower().endswith('.png')]
   
test_images = []

for filename in source_image_files:
    test_image = cv2.imread(os.path.join(test_folder, filename), cv2.IMREAD_GRAYSCALE)
    test_image = cv2.resize(test_image, (512, 512))
    #test_image = cv2.bitwise_not(test_image)
    test_image = test_image.astype('float32') / 255.0
    test_images.append(test_image)


test_images = np.array(test_images)
test_images = np.expand_dims(test_images, axis=-1)
print("Test Images Shape:", test_images.shape)

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, axs = plt.subplots(2, 5)
for i in range(2):
    for j in range(5):
      if i == 0:
            axs[i, j].imshow(source_images[j], cmap='gray')
            axs[i, j].set_title('Source image')
            axs[i, j].axis('off')
      else:
            axs[i, j].imshow(target_images[j], cmap='gray')
            axs[i, j].set_title('Target image')
            axs[i, j].axis('off')

plt.show()

# Autoencoder models for bone suppresion

## 400/100 Train images 1024x1024x1

### Using MSE

In [ ]:
#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():    
input_shape = (source_images.shape[1],source_images.shape[2], 1)

input_layer = Input(shape=input_shape)

# Encoder
encoder = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
#encoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
#encoder = MaxPooling2D((2, 2), padding='same')(encoder)

# Decoder
#decoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
#decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder) #Change to decoder if using code above
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoder)


model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss="mse", metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4


tf.debugging.set_log_device_placement(True)

try:
  with tf.device('/device:GPU:1'):
    train_gen = DataGenerator(X_train, y_train, batch_size)
    test_gen = DataGenerator(X_val, y_val, batch_size)
    history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=epochs,
    verbose=1,
    callbacks=[ShowReconstruction(), reduce_lr_callback],
    shuffle=True,
    workers=8
    )
except RuntimeError as e:
  print(e)

In [ ]:
model.save('AE_MSE_50epochs_400.h5')

In [ ]:
predicted_images = None
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

### Using MS-SSIM

In [ ]:
#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():    
input_shape = (source_images.shape[1],source_images.shape[2], 1)

input_layer = Input(shape=input_shape)

# Encoder
encoder = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)

# Decoder
decoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoder)


model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=ms_ssim_loss, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4


tf.debugging.set_log_device_placement(True)

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)
history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback],
shuffle=True,
workers=8
)

In [ ]:
model.save('AE_MSSSIM_50epochs_400.h5')

In [ ]:
predicted_images = None
model = load_model('AE_MSSSIM_50epochs_400.h5', custom_objects={'ms_ssim_loss' : ms_ssim_loss,'ms_ssim_metric' : ms_ssim_metric, 'psnr' : psnr})
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

In [ ]:
model.save('AE_MSSSIM.h5')

In [ ]:
predicted_images = None

model = load_model('AE_MSSSIM.h5', custom_objects={'ms_ssim_loss': ms_ssim_loss, 'ms_ssim_metric' : ms_ssim_metric})
predicted_images = model.predict(test_images)

num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

### Using Mix loss

In [ ]:
input_shape = (source_images.shape[1],source_images.shape[2], 1)

input_layer = Input(shape=input_shape)

# Encoder
encoder = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)

# Decoder
decoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoder)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4


tf.debugging.set_log_device_placement(True)

try:
  with tf.device('/device:GPU:1'):
    train_gen = DataGenerator(X_train, y_train, batch_size)
    test_gen = DataGenerator(X_val, y_val, batch_size)
    history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=epochs,
    verbose=1,
    callbacks=[ShowReconstruction(), reduce_lr_callback],
    shuffle=True,
    workers=8
    )
except RuntimeError as e:
  print(e)

In [ ]:
model.save('AE_MixLoss_50epochs_400.h5')

In [ ]:
try:
    with tf.device('/device:GPU:1'):
        model.evaluate(test_gen)
except Exception as e:
    print("An error occurred:", str(e))

In [ ]:
predicted_images = None
model = load_model('AE_MixLoss_50epochs_400.h5', custom_objects={'mixed_loss_l2': mixed_loss_l2, 'ms_ssim_metric' : ms_ssim_metric, 'psnr':psnr})
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## 1000/100 Train images 1024x1024x1

### Using MSE

In [ ]:
#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():    
input_shape = (source_images.shape[1],source_images.shape[2], 1)

input_layer = Input(shape=input_shape)

# Encoder
encoder = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
#encoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
#encoder = MaxPooling2D((2, 2), padding='same')(encoder)

# Decoder
#decoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
#decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoder)


model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss="mse", metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4


tf.debugging.set_log_device_placement(True)


train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[reduce_lr_callback], #ShowReconstruction() commented since the validation inference cause the GPU to OOM
shuffle=True,
workers=8
)

In [ ]:
model.save('AE_MSE_50epochs_1000.h5')

In [ ]:
predicted_images = None
model = load_model('AE_MSE_50epochs_1000.h5', custom_objects={'ms_ssim_metric' : ms_ssim_metric, 'psnr' : psnr})
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

### Using MS-SSIM

In [ ]:
#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():    
input_shape = (source_images.shape[1],source_images.shape[2], 1)

input_layer = Input(shape=input_shape)

# Encoder
encoder = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)

# Decoder
decoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoder)


model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=ms_ssim_loss, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4


tf.debugging.set_log_device_placement(True)

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)
history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback],
shuffle=True,
workers=8
)

In [ ]:
model.save('AE_MSSSIM_50epochs_1000.h5')

In [ ]:
model.evaluate(test_gen)

In [ ]:
predicted_images = None
model = load_model('AE_MSSSIM_50epochs_1000.h5', custom_objects={'ms_ssim_loss': ms_ssim_loss, 'ms_ssim_metric' : ms_ssim_metric, 'psnr':psnr})
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

### Using Mix loss

In [ ]:
input_shape = (source_images.shape[1],source_images.shape[2], 1)

input_layer = Input(shape=input_shape)

# Encoder
encoder = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)

# Decoder
decoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoder)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4


tf.debugging.set_log_device_placement(True)

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)
history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback],
shuffle=True,
workers=8
)

In [ ]:
model.save('AE_MixLoss_50epochs_1000train.h5')

In [ ]:
model.evaluate(test_gen)

In [ ]:
predicted_images = None
model = load_model('AE_MixLoss_50epochs_1000train.h5', custom_objects={'mixed_loss_l2': mixed_loss_l2, 'ms_ssim_metric' : ms_ssim_metric, 'psnr':psnr})
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## Full training set

### Using MSE

In [ ]:
#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():    
input_shape = (source_images.shape[1],source_images.shape[2], 1)

input_layer = Input(shape=input_shape)

# Encoder
encoder = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)

# Decoder
decoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoder)



model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss="mse", metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 8

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

try:
  with tf.device('/device:GPU:1'):
    history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=epochs,
    verbose=1,
    callbacks=[ShowReconstruction(), reduce_lr_callback]
    )
except RuntimeError as e:
  print(e)

In [ ]:
model.save('AE_MSE_50epochs_full.h5')

In [ ]:
model.evaluate(test_gen)

In [ ]:
predicted_images = None
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

### Using MS-SSIM

In [ ]:
#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():    
input_shape = (source_images.shape[1],source_images.shape[2], 1)

input_layer = Input(shape=input_shape)

# Encoder
encoder = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)

# Decoder
decoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoder)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=ms_ssim_loss, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 60
batch_size = 16

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback]
)


In [ ]:
model.save('AE_MSSIM_60epochs_full.h5')

In [ ]:
model.evaluate(test_gen)

In [ ]:
predicted_images = None
model = load_model('AE_MSSIM_60epochs_full.h5', custom_objects={'ms_ssim_loss': ms_ssim_loss, 'ms_ssim_metric' : ms_ssim_metric, 'psnr':psnr})
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

### Using Mix loss

In [ ]:
#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():    
input_shape = (source_images.shape[1],source_images.shape[2], 1)

input_layer = Input(shape=input_shape)

# Encoder
encoder = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)

# Decoder
decoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoder)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 60
batch_size = 16

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback]
)


In [ ]:
model.save('AE_MixLoss_60epochs_full.h5')

In [ ]:
predicted_images = None
#model = load_model('AE_MixLoss_50epochs_full.h5', custom_objects={'mixed_loss_l2': mixed_loss_l2, 'ms_ssim_metric' : ms_ssim_metric})
predicted_images = model.predict(test_images)

num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

### Using another Mix Loss

In [ ]:
#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():    
input_shape = (source_images.shape[1],source_images.shape[2], 1)

input_layer = Input(shape=input_shape)

# Encoder
encoder = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
#encoder = Conv2D(1024, (3, 3), activation='relu', padding='same')(encoder)
#encoder = MaxPooling2D((2, 2), padding='same')(encoder)

# Decoder
#decoder = Conv2D(1024, (3, 3), activation='relu', padding='same')(encoder)
#decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoder)


try:
  # Specify an invalid GPU device
  with tf.device('/device:GPU:1'):
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l1, metrics=["MSE", "MAE", ms_ssim_metric, psnr])
    model.summary()
except RuntimeError as e:
  print(e)

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback]
)

In [ ]:
model.save('AE_AdvMixLoss_100epochs_full.h5')

In [ ]:
#50epoch
try:
    with tf.device('/device:GPU:1'):
        model.evaluate(test_gen)
except Exception as e:
    print("An error occurred:", str(e))

In [ ]:
try:
    with tf.device('/device:GPU:1'):
        model.evaluate(test_gen)
except Exception as e:
    print("An error occurred:", str(e))

In [ ]:
predicted_images = None
#model = load_model('AE_MixLoss_50epochs_full.h5', custom_objects={'mixed_loss_l1': mixed_loss_l1, 'ms_ssim_metric' : ms_ssim_metric})
predicted_images = model.predict(test_images)

num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## Pre-trained backones

### Using ResNet

In [ ]:
def build_unet_with_resnet(input_size=(source_images.shape[1],source_images.shape[2],1)):
    input_img = Input(shape=input_size)

    # Expand the single-channel input to three channels
    expanded_input = tf.keras.layers.Lambda(lambda x: tf.concat([x, x, x], axis=-1))(input_img)

    # Pre-trained ResNet as the encoder
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(1024, 1024, 3))

    # Extract features from the ResNet model
    encoder_output = base_model(expanded_input)

    # Decoder part of the U-Net
    x = Conv2D(1024, (3, 3), activation='relu', padding='same')(encoder_output)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    #x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    #x = BatchNormalization()(x)
    #x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)
    return Model(inputs=input_img, outputs=decoded)

model = build_unet_with_resnet()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 48
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback]
)


In [ ]:
model.save('AE_Resnet_50epochs_full.h5')

In [ ]:
model.evaluate(test_gen)


In [ ]:
predicted_images = None
#model = load_model('UNET_Resnet_50epochs_full.h5', custom_objects={'mixed_loss_l2':mixed_loss_l2,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
predicted_images = model.predict(test_images)

num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

### Using EfficientNet

In [ ]:
def build_unet_with_effnet(input_size=(source_images.shape[1],source_images.shape[2],1)):
    input_img = Input(shape=input_size)

    # Expand the single-channel input to three channels
    expanded_input = tf.keras.layers.Lambda(lambda x: tf.concat([x, x, x], axis=-1))(input_img)

    # Pre-trained ResNet as the encoder
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(1024, 1024, 3))
    
    base_model.trainable = False

    # Extract features from the ResNet model
    encoder_output = base_model(expanded_input)

    # Decoder part of the U-Net
    x = Conv2D(1024, (3, 3), activation='relu', padding='same')(encoder_output)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    #x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    #x = BatchNormalization()(x)
    #x = UpSampling2D((2, 2))(x)
    # Output layer
    decoded = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)

    # Create the model
    return Model(inputs=input_img, outputs=decoded)

model = build_unet_with_effnet()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback]
)


In [ ]:
model.save('AE_EfficientNet_50epochs_full.h5')


In [ ]:
predicted_images = None
#model = load_model('UNET_Resnet_50epochs_full.h5', custom_objects={'mixed_loss_l2':mixed_loss_l2,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

### FPN Model with EfficientNet

In [ ]:
input_img = Input(shape=(1024, 1024, 1))
expanded_input = tf.keras.layers.Lambda(lambda x: tf.concat([x, x, x], axis=-1))(input_img)

# Pre-trained ResNet as the encoder
efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=expanded_input)

# FPN layers
C2 = efficientnet.get_layer('block2a_expand_activation').output
C3 = efficientnet.get_layer('block3a_expand_activation').output
C4 = efficientnet.get_layer('block4a_expand_activation').output
C5 = efficientnet.get_layer('block6a_expand_activation').output

# Top-down pathway
P5 = Conv2D(256, (1, 1), activation='relu', padding='same')(C5)
P4 = Conv2D(256, (1, 1), activation='relu', padding='same')(C4) + UpSampling2D()(P5)
P3 = Conv2D(256, (1, 1), activation='relu', padding='same')(C3) + UpSampling2D()(P4)
P2 = Conv2D(256, (1, 1), activation='relu', padding='same')(C2) + UpSampling2D()(P3)

P2 = UpSampling2D()(P2)

# Decoder layers
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(P2)  # Output should match the input shape (3 channels)


model = tf.keras.Model(inputs=input_img, outputs=decoded)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback]
)


In [ ]:
model.save('FPN_EfficientNet_100epochs_full.h5')


In [ ]:
model.evaluate(test_gen)


In [ ]:
predicted_images = None
#model = load_model('UNET_Resnet_50epochs_full.h5', custom_objects={'mixed_loss_l2':mixed_loss_l2,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

# U-Net architecture

## U-net default architecture | MSE loss

In [ ]:
def unet(input_size=(source_images.shape[1],source_images.shape[2],1)):
    inputs = Input(input_size)

    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = Concatenate()([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = Concatenate()([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = Concatenate()([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = Concatenate()([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    c10 = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    return tf.keras.Model(inputs=[inputs], outputs=[c10])

model = unet()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss="mse", metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
model = load_model('UNET_MSE_50epochs_full.h5', custom_objects={'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback]
)


In [ ]:
model.save('UNET_MSE_50epochs_full.h5')

In [ ]:
model.evaluate(test_gen)

In [ ]:
predicted_images = None
try:
    with tf.device('/device:GPU:1'):
        model = load_model('UNET_MSE_50epochs_full.h5', custom_objects={'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
        predicted_images = model.predict(test_images)
except Exception as e:
    print("An error occurred:", str(e))

num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## U-net default architecture | MS-SSIM

In [ ]:
def unet(input_size=(source_images.shape[1],source_images.shape[2],1)):
    inputs = Input(input_size)

    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = Concatenate()([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = Concatenate()([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = Concatenate()([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = Concatenate()([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    c10 = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    return tf.keras.Model(inputs=[inputs], outputs=[c10])

model = unet()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=ms_ssim_loss, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback]
)


In [ ]:
model.save('UNET_MSSIM_50epochs_full.h5')

In [ ]:
model.evaluate(test_gen)

In [ ]:
predicted_images = None
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## U-net default architecture | Mix loss

In [ ]:
def unet(input_size=(source_images.shape[1],source_images.shape[2],1)):
    inputs = Input(input_size)

    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = Concatenate()([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = Concatenate()([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = Concatenate()([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = Concatenate()([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    c10 = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    return tf.keras.Model(inputs=[inputs], outputs=[c10])

model = unet()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback]
)

In [ ]:
model.save('UNET_Mixloss_50epochs_full.h5')

In [ ]:
try:
    with tf.device('/device:GPU:1'):
        model.evaluate(test_gen)
except Exception as e:
    print("An error occurred:", str(e))

In [ ]:
predicted_images = None
try:
    with tf.device('/device:GPU:1'):
        model = load_model('UNET_Mixloss_50epochs_full.h5', custom_objects={'mixed_loss_l2':mixed_loss_l2,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
        predicted_images = model.predict(test_images)
except Exception as e:
    print("An error occurred:", str(e))

num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

# Segmentation models from pre-trained library

## SM-Unet | Mix loss

In [ ]:
model = sm.Unet('efficientnetb0', input_shape=(1024,1024, 1), encoder_weights=None)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

checkpoint_path = 'best_model.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min', verbose=1)

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback, checkpoint]
)


In [ ]:
model.save('Unet_SM_EfficientNet_50epochs_full.h5')


In [ ]:
model.evaluate(test_gen)


In [ ]:
predicted_images = None
#model = load_model('Unet_SM_EfficientNet_50epochs_full.h5', custom_objects={'mixed_loss_l2':mixed_loss_l2,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## SM-Unet | Another Mix loss

In [ ]:
model = sm.Unet('efficientnetb0', input_shape=(1024,1024, 1), encoder_weights=None)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l1, metrics=["MSE","MAE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

checkpoint_path = 'best_model.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min', verbose=1)

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback, checkpoint]
)


In [ ]:
model.save('Unet_SM_EfficientNet_adv_50epochs_full.h5')


In [ ]:
model.evaluate(test_gen)

In [ ]:
predicted_images = None
#model = load_model('Unet_SM_EfficientNet_adv_50epochs_full.h5', custom_objects={'loss_mix':loss_mix,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()
    
    
    predicted_image = cv2.normalize(predicted_image, None, 0, 255, cv2.NORM_MINMAX)
    predicted_image = np.uint8(predicted_image)
    predicted_image = cv2.bitwise_not(predicted_image)
    
    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

In [ ]:
predicted_images = None
try:
    with tf.device('/device:GPU:1'):
        model = load_model('AE_MixLoss_60epochs_full.h5', custom_objects={'mixed_loss_l1':mixed_loss_l1,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
        predicted_images = model.predict(test_images)
except Exception as e:
    print("An error occurred:", str(e))

num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## SM-FPN | Mix loss

In [ ]:
model = sm.FPN('resnet50', encoder_weights=None, activation="sigmoid", classes=1)
    
inp = Input(shape=(None, None, 1))
l1 = Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
out = model(l1)

model = Model(inp, out, name=model.name)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

checkpoint_path = 'best_model.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min', verbose=1)

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback, checkpoint]
)


In [ ]:
model.save('FPN_SM_ResNet50_50epochs_full.h5')


In [ ]:
try:
  # Specify an invalid GPU device
  with tf.device('/device:GPU:1'):
    model = load_model('FPN_SM_ResNet50_50epochs_full.h5', custom_objects={'mixed_loss_l2':mixed_loss_l2,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
    model.evaluate(test_gen)
except RuntimeError as e:
  print(e)

## SM-FPN | EfficientNet Mix Loss

In [ ]:
model = sm.FPN('efficientnetb0', encoder_weights=None, activation="sigmoid", classes=1)

inp = Input(shape=(None, None, 1))
l1 = Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
out = model(l1)

model = Model(inp, out, name=model.name)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

checkpoint_path = 'best_model.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min', verbose=1)

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback, checkpoint]
)


In [ ]:
model.save('FPN_SM_EfficientNet_50epochs_full.h5')


In [ ]:
try:
  # Specify an invalid GPU device
  with tf.device('/device:GPU:1'):
    model = load_model('FPN_SM_EfficientNet_50epochs_full.h5', custom_objects={'loss_mix':loss_mix,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
    model.evaluate(test_gen)
except RuntimeError as e:
  print(e)

In [ ]:
predicted_images = None
#model = load_model('FPN_SM_EfficientNet_50epochs_full.h5', custom_objects={'loss_mix':loss_mix,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()
    predicted_image_invert = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

# Segmentation model + inverted dataset

In [ ]:
source_folder = 'Dataset/source'
target_folder = 'Dataset/target'

# List all image file names in the source folder
source_image_files = os.listdir(source_folder)

# Initialize lists to store the preprocessed images
source_images = []
target_images = []

i = 0

# Loop through each image file in the source folder
for filename in source_image_files:
    
    # Load source image
    source_image = cv2.imread(os.path.join(source_folder, filename), cv2.IMREAD_GRAYSCALE)
    source_image = source_image.astype('float32') / 255.0
    source_images.append(source_image)

    # Load corresponding target image from the target folder
    target_image = cv2.imread(os.path.join(target_folder, filename), cv2.IMREAD_GRAYSCALE)
    target_image = target_image.astype('float32') / 255.0
    target_images.append(target_image)
    
    #Add bitwise inverted images:
    if i > 100: 
        source_image = cv2.imread(os.path.join(source_folder, filename), cv2.IMREAD_GRAYSCALE)
        source_image = cv2.bitwise_not(source_image)
        source_image = source_image.astype('float32') / 255.0
        source_images.append(source_image)

        target_image = cv2.imread(os.path.join(target_folder, filename), cv2.IMREAD_GRAYSCALE)
        target_image = cv2.bitwise_not(target_image)
        target_image = target_image.astype('float32') / 255.0
        target_images.append(target_image)
    
    # Resize images if needed (adjust the dimensions)
    #source_image = cv2.resize(source_image, (512, 512))
    #target_image = cv2.resize(target_image, (512, 512))
    
    i += 1
    #if i == 500:
    #    break

# Convert the lists to NumPy arrays
source_images = np.array(source_images)
target_images = np.array(target_images)

source_images = np.expand_dims(source_images, axis=-1)
target_images = np.expand_dims(target_images, axis=-1)
# Print the shape of the loaded and preprocessed images
print("Source Images Shape:", source_images.shape)
print("Target Images Shape:", target_images.shape)

In [ ]:
# Use first 100 as a test set
X_train = source_images[100:]
y_train = target_images[100:]

X_val = source_images[:100]
y_val = target_images[:100]

print(f"x_train: {len(X_train)} | x_val: {len(X_val)} | y_train: {len(y_train)} | y_val: {len(y_val)}")

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, axs = plt.subplots(2, 5)
for i in range(2):
    for j in range(5):
      if i == 0:
            axs[i, j].imshow(source_images[j], cmap='gray')
            axs[i, j].set_title('Source image')
            axs[i, j].axis('off')
      else:
            axs[i, j].imshow(target_images[j], cmap='gray')
            axs[i, j].set_title('Target image')
            axs[i, j].axis('off')



plt.show()

## FPN

In [ ]:
model = sm.FPN('efficientnetb0', encoder_weights=None, activation="sigmoid", classes=1)

inp = Input(shape=(None, None, 1))
l1 = Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
out = model(l1)

model = Model(inp, out, name=model.name)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback, checkpoint]
)


In [ ]:
model.evaluate(test_gen)


In [ ]:
predicted_images = None
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()
    predicted_image_invert = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## U-net Mix loss

In [ ]:
model = sm.Unet('efficientnetb0', input_shape=(1024,1024, 1), encoder_weights=None)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback, checkpoint]
)


In [ ]:
#After 50epochs
model.evaluate(test_gen)


In [ ]:
predicted_images = None
predicted_images = model.predict(test_images)

num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()
    predicted_image_invert = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

In [ ]:
model.save('Unet_SM_EfficientNet_100epochs_invertedfull.h5')


In [ ]:
#After 100epochs
model.evaluate(test_gen)


In [ ]:
predicted_images = None
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()
    predicted_image_invert = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## U-net | Advanced mix loss

In [ ]:
model = sm.Unet('efficientnetb0', input_shape=(1024,1024, 1), encoder_weights=None)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l1, metrics=["MSE", "MAE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback, checkpoint]
)


In [ ]:

model.save('Unet_SM_EfficientNet_adv_50epochs_inverted_full.h5')


In [ ]:
#After 50epochs
model.evaluate(test_gen)


In [ ]:
model.save('Unet_SM_EfficientNet_adv_100epochs_inverted_full.h5')


In [ ]:
#After 100epochs
model.evaluate(test_gen)


In [ ]:
predicted_images = None
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()
    predicted_image_invert = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

# Test of AE and U-net on 512x512x1

## AE + Mixed L1 loss

In [ ]:
#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():    
input_shape = (source_images.shape[1],source_images.shape[2], 1)

input_layer = Input(shape=input_shape)

# Encoder
encoder = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
encoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
#encoder = Conv2D(1024, (3, 3), activation='relu', padding='same')(encoder)
#encoder = MaxPooling2D((2, 2), padding='same')(encoder)

# Decoder
#decoder = Conv2D(1024, (3, 3), activation='relu', padding='same')(encoder)
#decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoder)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l1, metrics=["MSE", "MAE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[reduce_lr_callback] 
)

In [ ]:
model.save('AE_512_MixedL1Loss_50epochs_full.h5')

In [ ]:
model.evaluate(test_gen)

In [ ]:
predicted_images = None
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## FPN + EfficientNetB0

In [ ]:
input_img = Input(shape=(512, 512, 1))
expanded_input = tf.keras.layers.Lambda(lambda x: tf.concat([x, x, x], axis=-1))(input_img)

# Pre-trained ResNet as the encoder
efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=expanded_input)

# FPN layers
C2 = efficientnet.get_layer('block2a_expand_activation').output
C3 = efficientnet.get_layer('block3a_expand_activation').output
C4 = efficientnet.get_layer('block4a_expand_activation').output
C5 = efficientnet.get_layer('block6a_expand_activation').output

# Top-down pathway
P5 = Conv2D(256, (1, 1), activation='relu', padding='same')(C5)
P4 = Conv2D(256, (1, 1), activation='relu', padding='same')(C4) + UpSampling2D()(P5)
P3 = Conv2D(256, (1, 1), activation='relu', padding='same')(C3) + UpSampling2D()(P4)
P2 = Conv2D(256, (1, 1), activation='relu', padding='same')(C2) + UpSampling2D()(P3)

P2 = UpSampling2D()(P2)

# Decoder layers
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(P2) 

model = tf.keras.Model(inputs=input_img, outputs=decoded)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[reduce_lr_callback] 
)


In [ ]:
model.save('FPN_512_EfficientNet_50epochs_full.h5')

In [ ]:
model.evaluate(test_gen)

In [ ]:
predicted_images = None
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## U-net + Another Mixed Loss

In [ ]:
def unet(input_size=(source_images.shape[1],source_images.shape[2],1)):
    inputs = Input(input_size)

    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = Concatenate()([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = Concatenate()([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = Concatenate()([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = Concatenate()([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    c10 = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    return tf.keras.Model(inputs=[inputs], outputs=[c10])

model = unet()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback]
)

In [ ]:
model.save('UNET_512_Mixloss_50epochs_full.h5')

In [ ]:
model.evaluate(test_gen)

In [ ]:
predicted_images = None
try:
    with tf.device('/device:GPU:1'):
        model = load_model('UNET_Mixloss_50epochs_full.h5', custom_objects={'mixed_loss_l2':mixed_loss_l2,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})
        predicted_images = model.predict(test_images)
except Exception as e:
    print("An error occurred:", str(e))

num_samples_to_visualize = 5
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[idx].squeeze()
    predicted_image = predicted_images[idx].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()

## SM U-net + Mixed L2 loss

In [ ]:
model = sm.Unet('efficientnetb0', input_shape=(512,512, 1), encoder_weights=None) 
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mixed_loss_l2, metrics=["MSE", ms_ssim_metric, psnr])
model.summary()

In [ ]:
epochs = 50
batch_size = 4

train_gen = DataGenerator(X_train, y_train, batch_size)
test_gen = DataGenerator(X_val, y_val, batch_size)

tf.debugging.set_log_device_placement(True)

history = model.fit(
train_gen,
validation_data=test_gen,
epochs=epochs,
verbose=1,
callbacks=[ShowReconstruction(), reduce_lr_callback, checkpoint]
)

In [ ]:
model.save('Unet_512_SM_EfficientNet_50epochs_full.h5')

In [ ]:
model = load_model('Unet_512_SM_EfficientNet_50epochs_full.h5', custom_objects={'mixed_loss_l2':mixed_loss_l2,'psnr': psnr, 'ms_ssim_metric' : ms_ssim_metric})

In [ ]:
test_gen = DataGenerator(X_val, y_val, 4)
model.evaluate(test_gen)

In [ ]:
predicted_images = None
predicted_images = model.predict(test_images)


num_samples_to_visualize = 6
random_indices = np.random.choice(len(test_images), num_samples_to_visualize)

for i, idx in enumerate(random_indices):
    original_image = test_images[i].squeeze()
    predicted_image = predicted_images[i].squeeze()

    plt.figure(figsize=(12, 6))


    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap='gray')


    plt.subplot(1, 2, 2)
    plt.title("Predicted Image")
    plt.imshow(predicted_image, cmap='gray')

    plt.tight_layout()
    plt.show()